# EyePredict CNN

In [6]:
import numpy as np
import pandas as pd 
import pickle
import math
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf

from random import shuffle
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [7]:
def load_data():
    print("log....Loading the data" )
    with open('labels_data.pickle', 'rb') as handle:
        labelsData = pickle.load(handle)
    with open('samples_data.pickle', 'rb') as handle:
        samplesData = pickle.load(handle)
        
    #create one df for shuffel
    allData = pd.DataFrame(
        {'samples_data': samplesData,
         'labels_data': labelsData
        })
    #shuffel the data
    allData = allData.sample(frac=1).reset_index(drop=True)

    #separate the data to samples and lables again
    X = allData.samples_data
    y = allData.labels_data

    #flatten the samples
    flattendX = []
    for sample in X:
        flattendX.append(sample.flatten()) 

    # test_size: what proportion of original data is used for test set
    train_sample, test_sample, train_lbl, test_lbl = train_test_split(
        flattendX, y, test_size=1/7.0, random_state=0)

    train_sample = np.float32(np.asarray(train_sample))
    test_sample = np.float32(np.asarray(test_sample))
    train_lbl = np.int32(np.asarray(train_lbl))
    test_lbl = np.int32(np.asarray(test_lbl))
    print("log....Done prepering the data" )
    return train_sample, test_sample, train_lbl, test_lbl

In [8]:
def standardize_data(train_data, test_data):
    #Standardize the Data
    scaler = StandardScaler()
    # Fit on training set only.
    scaler.fit(train_data)
    # Apply transform to both the training set and the test set.
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)
    
    return train_data, test_data

In [9]:
def pca_transform(train_data, test_data):
    # Make an instance of the Model
    pca = PCA(.95)
    
    #Apply the mapping (transform) to both the training set and the test set
    pca.fit(train_data)
    
    train_data = pca.transform(train_data)
    test_data = pca.transform(test_data)
    
    return train_data, test_data

In [10]:
def data_reshape(data):
    #Add Padding to the samples to be on size 3600
    data_resized_list = []
    total_pad_to_add = 3600 - data.shape[1]
    pad_to_each_axes = total_pad_to_add/2
    x_axis = math.floor(pad_to_each_axes)
    y_axis = math.ceil(pad_to_each_axes)
    for sample in data:
        tempSample = np.pad(sample, (x_axis,y_axis), 'constant', constant_values=(0, 0))
        sqrtShape = int(math.sqrt(tempSample.shape[0]))
        data_resized_list.append(tempSample.reshape(sqrtShape,sqrtShape))
    return np.asarray(data_resized_list), sqrtShape

In [11]:
def logistic_regresion(features, labels, mode):


SyntaxError: unexpected EOF while parsing (<ipython-input-11-49b6af507ce2>, line 1)

## Load Data

In [ ]:
# Load training and eval data
train_data, eval_data, train_labels, eval_labels = load_data()

## Standardize The Data

In [ ]:
train_labels = train_labels - 1
eval_labels = eval_labels - 1

In [ ]:
#standardize
train_data, eval_data = standardize_data(train_data, eval_data)

## Run PCA Transform

In [ ]:
#pca
train_data, eval_data = pca_transform(train_data, eval_data)

## Reshape The Data To fit Tensor

In [ ]:
#padding
train_data, shape = data_reshape(train_data)
eval_data, shape = data_reshape(eval_data)

In [ ]:
for data in eval_labels:
    if np.any(np.isnan(data)):
        print('NAN')

## Run LR Model